# σ-加法族からホップの拡張定理まで

今回の内容は [測度・確率・ルベーグ積分 応用への最短コース](https://www.amazon.co.jp/%E6%B8%AC%E5%BA%A6%E3%83%BB%E7%A2%BA%E7%8E%87%E3%83%BB%E3%83%AB%E3%83%99%E3%83%BC%E3%82%B0%E7%A9%8D%E5%88%86-%E5%BF%9C%E7%94%A8%E3%81%B8%E3%81%AE%E6%9C%80%E7%9F%AD%E3%82%B3%E3%83%BC%E3%82%B9-KS%E7%90%86%E5%B7%A5%E5%AD%A6%E5%B0%82%E9%96%80%E6%9B%B8-%E5%8E%9F-%E5%95%93%E4%BB%8B/dp/4061565710) を参考にしています。


# σ-加法族

``σ-加法族``や``測度``という概念は、無限回の試行を考える確率論を導入するために重要な概念です。
逆に、こういうややこしい概念を考えないと、確率の議論に矛盾が生じることがあります。

例として、次のような問題を考えてみます。
これは[Pillow Problems](https://www.amazon.com/gp/product/0486204936?ie=UTF8&tag=alfuanga04-20&linkCode=as2&camp=1789&creative=390957&creativeASIN=0486204936)にある問題58を少し簡易化したものです。

---
**例題: 無限平面上の点**

無限平面上に点を1つランダムに選ぶ。これが第一象限に入っている確率を求めよ。

平面上の点は第一から第四象限のどれかにはいるはずなので、ランダムに点を選択するならば、確率は $1/4$ になりそうです。
これは本当に正しいでしょうか？実は、この問題自体におかしな仮定があります。

そもそも``「無限平面上にランダムに点を選ぶ」``ができません。
ある正方形 $A$ に点が含まれる確率を考えてみましょう。
どこでも平等に点を選ぶため、別の場所にある同じ形の正方形 $B$ に点が含まれる確率と $A$ に点が含まれる確率は同じであり、２つのどちらかに入る確率はその和になります。しかし、無限平面には無限個の正方形を敷き詰めて良いので、確率が $0$ でない限り、平面のどこかに一点を選ぶ確率が無限大になります。
よって、無限平面上に``一様な``確率を定義してはいけません。

---

このように、無限の世界で確率を考える場合は、直感的な確率で考えると議論に矛盾が生じる場合があります。
一方で、例えば原点を中心とした有限な正方形内の格子点からランダムに点を選択することを考えれば、$1/4$ は正しい答えになります。
では、有限の世界の性質を持ったまま、無限の世界に確率を拡張させることはできるでしょうか？

コルモゴロフの[公理的確率](https://ja.wikipedia.org/wiki/%E7%A2%BA%E7%8E%87%E3%81%AE%E5%85%AC%E7%90%86)に基づいた確率の定義、
そして後で見るホップの拡張定理は、この問題を解決してくれます。

ルベーグ積分やσ-加法族、そして測度の概念は、こうした無限の世界への議論をするために必要になります。

---
**定義：σ-加法族**

$\Omega$を空でない集合とし, $\mathcal{F}$ を $\Omega$ の部分集合族 $\mathcal{F} \subset 2^\Omega$ とする。
次の性質を満たすとき、$\mathcal{F}$ を $\Omega$ 上のσ-加法族という。

1. $\emptyset \in \mathcal{F}$ 
2. $A \in \mathcal{F}$ ならば $A^c \in \mathcal{F}$
3. $A_1, A_2, \dots \in \mathcal{F}$ に対して、$\cup^\infty_{n=1} A_n \in \mathcal{F}$ 
---

集合と部分集合族に対して、そのσ-加法性をチェックしてみましょう ([参考](https://qiita.com/sasakinzm/items/429a9fc629aa04880a02))

In [1]:
from itertools import product

def check_sigma_algebra(outcomes: frozenset, family: frozenset):
    # 空集合の確認
    if frozenset([]) not in family:
        return False

    # 補集合の確認
    for _set in family:
        if outcomes.difference(_set) not in family:
            return False

    # 可算和の確認
    for n in range(len(family)):
        # A_n の 取り方を全て試す
        for A1_An in product(family, repeat=n):
            sum_A1_An = frozenset([])
            for Ai in A1_An:
                sum_A1_An = sum_A1_An.union(Ai)
            if sum_A1_An not in family:
                return False
    return True

In [2]:
Omega = frozenset(["a", "b"])  # 標本空間
F = frozenset([
    Omega, 
    frozenset([]),
    frozenset(["a"]),
    frozenset(["b"]),
])  # σ代数
assert check_sigma_algebra(Omega, F)


Omega = frozenset(["a", "b", "c"])  # 標本空間
F = frozenset([
    Omega, 
    frozenset([]),
    frozenset(["a"]),
    frozenset(["b", "c"]),
])  # σ代数
assert check_sigma_algebra(Omega, F)


Omega = frozenset(["a", "b", "c"])  # 標本空間
F = frozenset([
    Omega, 
    frozenset([]),
    frozenset(["a"]),
    frozenset(["b"]),
    frozenset(["b", "c"]),
])  # σ代数ではない

assert not check_sigma_algebra(Omega, F)

**σ-加法族の性質**

σ-加法族には、``可算個の演算については通常必要な操作をすべて許している`` という重要な性質があります。

実際、定義より、σ-加法族は和集合の操作について閉じています（可算有限・無限個の和）。
また、補集合の操作について閉じているので、積集合や差の操作についても閉じています ($A \cap B = (A^c\cup B^c)^c$, $A \setminus B = A\cap B^c$)。
この性質がコロモゴロフの公理に従った確率を定義する上で非常に重要になります。


**なぜ「σ」-「加法族」なの？**

σ-加法族のσは、数学の分野では可算無限に関連する用語として使用されます （[参考](https://tor-lattimore.com/downloads/book/book.pdf)のノート2.7.1）。
上述したように可算無限和（加法）について閉じているため、σ-``加法``族と呼ばれます。
また、可算無限の代数計算についても閉じているので、σ-``加法``族はσ-``代数``とも呼ばれます。


# 測度

このような``良い``性質をもったσ-加法族に対して、コロモゴロフの公理に従った確率を定義することができます。
コロモゴロフの公理は直感的には、「事象の上に``面積``のようなものが定義できて、その``面積の和が1`` になってたらその面積が確率」 という確率の定義を与えています。

先程のσ-加法族は``面積のようなものを定義できる事象``の集合を与えます。
そして、次の``測度``が一般的な面積のようなものを与えます （和が $1$ とは限りません）。

---
**定義：測度, 測度空間**

可測空間 $(\Omega, \mathcal{F})$ に対し, $\mathcal{F}$ 上で定義された関数 $\mu$ が以下の性質を満たす時、$\mu$ を $(\Omega, \mathcal{F})$ 上の測度という。

1. 任意の$A \in \mathcal{F}$ に対して, $0 \geq \mu(A) \geq \infty$。 特に $\mu(\empty) = 0$。
2. $A_1, A_2, \dots \in \mathcal{F}$ が非交差的ならば、$\mu(\sqcup^\infty_{n=1} A_n) = \sum^\infty_{n=1}\mu (A_n)$。

このとき、$(\Omega, \mathcal{F}, \mu)$ の組を測度空間と呼ぶ。

---

この $(\Omega, \mathcal{F})$ は $\mu$ によって面積が``測れる``ので``可測空間``と呼びます。
（なぜ面積なのか考えてみましょう。$\mathcal{F}$の要素を図形とみなせば、上の定義では、たかだか可算個の重なりの無い図形を合わせた図形の面積が、それぞれの面積の和になっています。これは直感的にも面積っぽいですね。）

測度を定義すれば、確率空間は直ちに定義されます。


---
**定義：確率空間, 確率**

以下の条件を満たす $(\Omega, \mathcal{F}, P)$ を確率空間と呼び、その $P$ を確率測度もしくは単に確率という。

1. $(\Omega, \mathcal{F}, P)$ は測度空間である。
2. $P(\Omega) = 1$ である

---

σ-加法族の場合と同様にして、確率空間をチェックしてみます。

In [15]:
from itertools import product
from typing import Callable
import numpy as np


def check_measurable_space(family: frozenset, measure: Callable[[frozenset], float]):
    for _set in family:
        # 1の確認
        if measure(_set) < 0:
            return False

    # 2の確認
    for n in range(len(family)):
        # A_n の 取り方を全て試す
        for A1_An in product(family, repeat=n):
            # 非交差になっている取り方について、測度を計算する
            is_disjoint = True
            sum_A1_An = frozenset([])
            sum_measure = 0

            for Ai in A1_An:
                is_disjoint = is_disjoint and sum_A1_An.isdisjoint(Ai)
                sum_A1_An = sum_A1_An.union(Ai)
                sum_measure += measure(Ai)

            if not is_disjoint:
                continue
            else:
                if not np.isclose(measure(sum_A1_An), sum_measure):
                    return False
    return True


def check_probability_space(outcomes: frozenset, family: frozenset, measure: Callable[[frozenset], float]):
    # 和が1の確認
    if measure(outcomes) != 1:
        return False

    if not check_measurable_space(family, measure):
        return False

    return True


In [16]:
Omega = frozenset(["a", "b"])  # 標本空間
F = frozenset([
    Omega, 
    frozenset([]),
    frozenset(["a"]),
    frozenset(["b"]),
])  # σ代数

def measure(_set: frozenset):
    return {
        Omega: 1.0,
        frozenset([]): 0.0, 
        frozenset(["a"]): 0.3, 
        frozenset(["b"]): 0.7
    }[_set]  # 確率空間である

assert check_measurable_space(F, measure)
assert check_probability_space(Omega, F, measure)


Omega = frozenset(["a", "b", "c"])  # 標本空間
F = frozenset([
    Omega, 
    frozenset([]),
    frozenset(["a"]),
    frozenset(["b", "c"]),
])  # σ代数

def measure(_set: frozenset):
    return {
        Omega: 1.0, 
        frozenset([]): 0.0,
        frozenset(["a"]): 0.9,
        frozenset(["b", "c"]): 0.1,
    }[_set]  # 確率空間である

assert check_measurable_space(F, measure)
assert check_probability_space(Omega, F, measure)



Omega = frozenset(["a", "b", "c"])  # 標本空間
F = frozenset([
    Omega, 
    frozenset([]),
    frozenset(["a"]),
    frozenset(["b"]),
    frozenset(["b", "c"]),
])  # σ代数ではない

def measure(_set: frozenset):
    return {
        Omega: 1.0, 
        frozenset([]): 0.0,
        frozenset(["a"]): 0.2,
        frozenset(["b"]): 0.3,
        frozenset(["b", "c"]): 0.5,
    }[_set]  # 確率空間ではない

assert not check_measurable_space(F, measure)
assert not check_probability_space(Omega, F, measure)

無限の世界での確率を扱うために色々定義してきましたが、この定義だけでは無限の世界にσ-加法族や確率測度を構築できるかは自明ではありません。
次の例を考えてみます。

---

**例：無限回のコイン投げ**

無限回コインを投げる施行について考えてみましょう。
つまり、標本 $\omega \in \Omega$ は「表」か「裏」の無限列になります。ここに自然なσ-加法族と確率測度を構築できるでしょうか？
この標本空間は無限集合であり、また、各事象は一様な確率を持つはずなので、最初のPillow Problemと同様の問題が生じます。
つまり、適当に確率空間を構築すると、確率の和が無限大になってしまいます。

---

無限の世界に確率測度を定義するにはどうすればよいでしょうか？
自然なアプローチとして、有限の場合の測度を構築しておいて、その性質を保つように無限の世界に測度を``拡張する``アプローチが考えられます。

測度の拡張についての正確な定義を与えておきます。

---

**定義：測度の拡張**
空出ない集合 $\Omega$ の 部分集合の集合族 $\mathcal{G}$ とその上の集合関数 $\mu: \mathcal{G} \to [0, \infty]$ を定義する。
ここで、$\mathcal{G}$ はσ-加法族でなくてもよく、$\mu$ も測度でなくても良い。

ここで、$\Omega$ 上のσ-加法族 $\mathcal{F}$ と測度 $\eta$ が存在し、$\mathcal{G} \subset \mathcal{F}$ であって、
任意の$A \in \mathcal{G}$ で $\mu(A) = \eta (A)$ のとき、「$\mu$ は測度 $\eta$に拡張される」 という。

---

ややこしいですね。Pythonで確認してみましょう。

In [18]:
Omega = frozenset(["a", "b", "c"])  # 標本空間

G = frozenset([
    Omega, 
    frozenset([]),
    frozenset(["a"]),
    frozenset(["b"]),
    frozenset(["b", "c"]),
])  # σ代数ではない

def mu(_set: frozenset):
    return {
        Omega: 0.3, 
        frozenset([]): 0.0,
        frozenset(["a"]): 0.1,
        frozenset(["b"]): 0.1,
        frozenset(["b", "c"]): 0.2,
    }[_set]  # 測度ではない

F = G.union(
    frozenset([
        frozenset(["c"]),
        frozenset(["a", "c"]),
        frozenset(["a", "b"]),
    ])
)  # Gを含むσ代数


def eta(_set: frozenset):
    return {
        # >>>> ここから G上の集合関数 >>>>
        Omega: 0.3, 
        frozenset([]): 0.0,
        frozenset(["a"]): 0.1,
        frozenset(["b"]): 0.1,
        frozenset(["b", "c"]): 0.2,
        # <<<< ここまで G上の集合関数 <<<<
        frozenset(["c"]): 0.1,
        frozenset(["a", "c"]): 0.2,
        frozenset(["a", "b"]): 0.2,
    }[_set]  # muを拡張した測度

assert check_measurable_space(F, eta)

上のPythonコードは「有限の世界から有限の世界に測度を拡張してみた例」です。では、有限から無限に測度は拡張できるのでしょうか？

この問題を解決するのが、ホップの拡張定理です。

---

**定理：ホップの拡張定理**

空でない集合 $\Omega$ 上の有限加法族 $\mathcal{F}$ 上の有限加法的測度 $\mu$ は、$\mu$ が $\mathcal{F}$ 上でσ-加法的であれば、$\mathcal{F}$ から生成されたσ-加法族 $\sigma(\mathcal{F})$ 上の測度 $\eta$ に拡張できる。さらに、$(\Omega, \mathcal{F}, \mu)$がσ-有限であれば、この拡張された$\eta$は一意である。

---

「生成されたσ-加法族」や「有限加法的測度」の定義については教科書に任せます。
これを使って、無限回のコイン投げの測度を構築してみます。

---

**例：無限回のコイン投げ**

標本空間を $\Omega = \{(\omega_n)_{n\in \mathbb{N}} : \omega_n \in \{表, 裏\}\}$ とします。つまり、「表」か「裏」の無限列です。

ここで、ホップの拡張定理を使うために、$\Omega$の有限な部分集合を考え、その有限加法族を構築します （具体的には、無限列に対して有限この場所とその結果は指定するけれど、それ以外は任意とする部分集合を考えます）。

任意の自然数$k$ と任意の自然数 $n_1 < n_2 < \dots < n_k$ と $\{裏, 表\}$の要素 $\varepsilon_1, \dots, \varepsilon_k$ に対して、

$$A(n_1, \dots, n_k; \varepsilon_1, \dots, \varepsilon_k) = \{(\omega_n)_{n\in \mathbb{N}}\in \Omega : \omega_{n_1} = \varepsilon_1, \dots, \omega_{n_k}=\varepsilon_k\}$$

とすれば、その有限和集合 $\mathcal{F}$ は明らかに有限加法族を構築します。そして、表が出る確率を $p$、$k$回目までに出る表の回数を$m$として、$\mathcal{F}$ 上の集合関数 $P$を次のように定義します。

$$P(A(n_1, \dots, n_k; \varepsilon_1, \dots, \varepsilon_k)) = p^m (1 - p)^{k-m}$$

これは明らかに $\mathcal{F}$上の有限加法的測度になっています。これが$\mathcal{F}$の上でσ-加法的であれば、ホップの定理から確率空間を一意に構築することができます。

拡張された確率空間の測度 $P'$ は $P$ が有限の世界で定義した性質をそのまま引き継いでいます。例えば、一回目の試行ででる確率は $P'=P=1/2$ になります。

---